# Lab | Chains in LangChain

## Outline

* LLMChain
* Sequential Chains
  * SimpleSequentialChain
  * SequentialChain
* Router Chain

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

GROQ_API_KEY  = os.getenv('GROQ_API_KEY')
HUGGINGFACEHUB_API_TOKEN = os.getenv('HUGGINGFACEHUB_API_TOKEN')

In [3]:
#!pip install pandas

In [4]:
import pandas as pd
df = pd.read_csv('./data/Data.csv')

In [5]:
df.head()

,Product,Review
0,Queen Size Sheet Set,I ordered a king size set. My only criticism w...
1,Waterproof Phone Pouch,"I loved the waterproof sac, although the openi..."
2,Luxury Air Mattress,This mattress had a small hole in the top of i...
3,Pillows Insert,This is the best throw pillow fillers on Amazo...
4,Milk Frother Handheld\r\n,I loved this product. But they only seem to l...


## LLMChain

In [ ]:
#!pip install langchain_community

In [7]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains import LLMChain

In [9]:
#Replace None by your own value and justify
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY, temperature=0.7)

I decided to use temperature 0.7 because it strikes a good balance between creativity and consistency — high enough to generate varied and interesting product descriptions, but not so high that the output becomes incoherent or off-topic.

In [10]:
prompt = ChatPromptTemplate.from_template(
    "Write a detailed and engaging description for the following product: {product}"
)

In [11]:

chain = LLMChain(llm=llm, prompt=prompt)

C:\Users\User\AppData\Local\Temp\ipykernel_45400\546483037.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use `RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt)


In [12]:
product = "perfume" #Select a product type to be describe
chain.run(product)

C:\Users\User\AppData\Local\Temp\ipykernel_45400\3980217193.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  chain.run(product)


"**Introducing the Essence of Elegance: Our Exquisite Perfume**\n\nImmerse yourself in a world of sophistication and refinement with our captivating perfume, carefully crafted to evoke the senses and leave a lasting impression. This luxurious fragrance is a masterful blend of the finest ingredients, expertly combined to create a scent that is both alluring and unforgettable.\n\nAs the delicate glass bottle is lifted, the intricate dance of aromas begins. The initial top notes of bergamot and lemon zest burst forth, releasing a vibrant and uplifting energy that sets the tone for a truly exceptional fragrance experience. The citrusy freshness is perfectly balanced by the subtle sweetness of floral notes, including rose and jasmine, which add a touch of femininity and elegance to the scent.\n\nAs the perfume settles, the heart of the fragrance is revealed, with rich and velvety smooth notes of vanilla and sandalwood emerging to create a sense of warmth and intimacy. The base notes of musk

## SimpleSequentialChain

In [13]:
from langchain_classic.chains import SimpleSequentialChain

In [15]:
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY, temperature=0.9)

# prompt template 1
first_prompt = ChatPromptTemplate.from_template(
    "Write a detailed and engaging description for the following product: {product}"
)

# Chain 1
chain_one = LLMChain(llm=llm, prompt=first_prompt)

In [16]:

# prompt template 2
second_prompt = ChatPromptTemplate.from_template(
    "Based on the following product description, write a short and catchy marketing tagline:\n\n{text}"
)
# chain 2
chain_two = LLMChain(llm=llm, prompt=second_prompt)

In [17]:
overall_simple_chain = SimpleSequentialChain(chains=[chain_one, chain_two],
                                             verbose=True
                                            )

In [18]:
overall_simple_chain.run(product)



> Entering new SimpleSequentialChain chain...
**Introducing "Eternal Bloom" - A Captivating Perfume Experience**

Imagine yourself surrounded by a garden of vibrant flowers, where the sweet scent of blooming blossoms fills the air and lifts your spirit. "Eternal Bloom" is a luxurious perfume that captures the essence of this idyllic scene, transporting you to a world of elegance and sophistication.

This exquisite fragrance is a masterful blend of top notes, including bergamot and mandarin, which provide a citrusy and uplifting introduction to the scent. As the fragrance unfolds, the heart notes of rose and jasmine emerge, releasing a floral explosion that is both alluring and seductive. The base notes of vanilla, sandalwood, and musk add warmth and depth to the perfume, leaving a lasting impression that lingers on the skin.

**A Symphony of Scents**

"Eternal Bloom" is a complex and alluring fragrance that evolves throughout the day, revealing new facets of its personality with each

'Here are a few short and catchy marketing tagline options:\n\n1. "Bloom into Beauty"\n2. "Eternal Elegance in Every Drop"\n3. "Unleash Your Inner Bloom"\n4. "Where Every Day is a Floral Fantasy"\n5. "Forever in Bloom, Forever in Style"\n\nBut my top pick would be:\n\n**"Bloom Forever"**\n\nThis tagline is short, memorable, and encapsulates the essence of the perfume\'s name and theme. It also implies that the fragrance will make the wearer feel beautiful and confident forever, which is a compelling message for potential customers.'

**Repeat the above twice for different products**

## SequentialChain

In [19]:
from langchain_classic.chains import SequentialChain

In [20]:
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY, temperature=0.9)


first_prompt = ChatPromptTemplate.from_template(
    "Translate the following review to English:\n\n{Review}"
)

chain_one = LLMChain(llm=llm, prompt=first_prompt, 
                     output_key="English_Review"
                    )

In [ ]:
second_prompt = ChatPromptTemplate.from_template(
    "Summarize the following review in one concise paragraph:\n\n{English_Review}"
)

chain_two = LLMChain(llm=llm, prompt=second_prompt, 
                        output_key="summary"
                    )

In [ ]:
# prompt template 3: detect the language of the original review
third_prompt = ChatPromptTemplate.from_template(
    "What language is the following review written in? Reply with just the language name.\n\n{Review}"
)
# chain 3: input= Review and output= language
chain_three = LLMChain(llm=llm, prompt=third_prompt,
                            output_key="language"
                      )

In [23]:

# prompt template 4: follow up message based on the summary and language
fourth_prompt = ChatPromptTemplate.from_template(
    "Write a short follow-up response to the customer based on the following summary: {summary}"
    "\n\nWrite the response in the original language: {language}"
)
chain_four = LLMChain(llm=llm, prompt=fourth_prompt,
                      output_key="followup_message"
                     )

In [24]:
# overall_chain: input= Review 
# and output= English_Review, summary, followup_message
overall_chain = SequentialChain(
    chains=[chain_one, chain_two, chain_three, chain_four],
    input_variables=["Review"],
    output_variables=["English_Review", "summary", "followup_message"],
    verbose=True
)

In [ ]:
review = df.Review[5]
overall_chain({"Review": review})

C:\Users\User\AppData\Local\Temp\ipykernel_45400\1992003631.py:2: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain-classic 0.1.0 and will be removed in 1.0. Use `invoke` instead.
  overall_chain(review)




> Entering new SequentialChain chain...

> Finished chain.


{'Review': "Je trouve le goût médiocre. La mousse ne tient pas, c'est bizarre. J'achète les mêmes dans le commerce et le goût est bien meilleur...\r\nVieux lot ou contrefaçon !?",
 'English_Review': 'Here is the translation of the review to English:\n\n"I find the taste mediocre. The foam doesn\'t last, it\'s weird. I buy the same ones in stores and the taste is much better... Old batch or counterfeit!?"\n\nNote: The reviewer is expressing their disappointment with the product, suggesting that it may be an old or counterfeit item, which is why it doesn\'t taste as good as the ones they buy in stores.',
 'summary': 'The reviewer is disappointed with the product, finding its taste to be mediocre and the foam to be short-lived. They note that the same product purchased in stores has a much better taste, leading them to suspect that the item they received may be either an old batch or counterfeit, which could explain the discrepancy in quality.',
 'followup_message': "Cher client,\n\nJe su

**Repeat the above twice for different products or reviews**

In [40]:
# Repeat 1: different review
review_2 = df.Review[1]
overall_chain({"Review": review_2})



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'I loved the waterproof sac, although the opening was made of a hard plastic. I don’t know if that would break easily. But I couldn’t turn my phone on, once it was in the pouch.',
 'English_Review': 'The review is already in English. However, I can provide a minor revision to make it clearer and more polished:\n\n"I loved the waterproof bag, although the opening was made of hard plastic, which made me wonder if it would break easily. However, I had an issue with it - once my phone was inside the pouch, I couldn\'t turn it on."',
 'summary': "The reviewer liked the waterproof bag but had two concerns: the hard plastic opening seemed fragile and might break easily, and they experienced an issue where they couldn't turn on their phone while it was inside the pouch.",
 'followup_message': "Dear valued customer,\n\nThank you for taking the time to share your thoughts about our waterproof bag. We're glad to hear that you liked the product overall. However, we apologize for the con

In [41]:
# Repeat 2: another review
review_3 = df.Review[3]
overall_chain({"Review": review_3})



> Entering new SequentialChain chain...

> Finished chain.


{'Review': 'This is the best throw pillow fillers on Amazon. I’ve tried several others, and they’re all cheap and flat no matter how much fluffing you do. Once you toss these in the dryer after you remove them from the vacuum sealed shipping material, they fluff up great',
 'English_Review': 'This review is already in English. However, I can provide a polished version of the review:\n\n"This is the best throw pillow filler available on Amazon. I\'ve tried several other options, but they were all cheap and flat, no matter how much I fluffed them. However, once I tossed these in the dryer after removing them from the vacuum-sealed packaging, they fluffed up perfectly."',
 'summary': "The reviewer highly recommends this throw pillow filler, stating it's the best available on Amazon. After trying other options that were cheap and flat, they were impressed with this product's ability to fluff up perfectly after a quick dry in the dryer, exceeding their expectations and outperforming its com

## Router Chain

In [26]:
physics_template = """You are a very smart physics professor. \
You are great at answering questions about physics in a concise\
and easy to understand manner. \
When you don't know the answer to a question you admit\
that you don't know.

Here is a question:
{input}"""


math_template = """You are a very good mathematician. \
You are great at answering math questions. \
You are so good because you are able to break down \
hard problems into their component parts, 
answer the component parts, and then put them together\
to answer the broader question.

Here is a question:
{input}"""

history_template = """You are a very good historian. \
You have an excellent knowledge of and understanding of people,\
events and contexts from a range of historical periods. \
You have the ability to think, reflect, debate, discuss and \
evaluate the past. You have a respect for historical evidence\
and the ability to make use of it to support your explanations \
and judgements.

Here is a question:
{input}"""


computerscience_template = """ You are a successful computer scientist.\
You have a passion for creativity, collaboration,\
forward-thinking, confidence, strong problem-solving capabilities,\
understanding of theories and algorithms, and excellent communication \
skills. You are great at answering coding questions. \
You are so good because you know how to solve a problem by \
describing the solution in imperative steps \
that a machine can easily interpret and you know how to \
choose a solution that has a good balance between \
time complexity and space complexity. 

Here is a question:
{input}"""

biology_template = """You are an excellent biologist. \
You have a deep understanding of living organisms, \
from the molecular and cellular level to entire ecosystems. \
You are skilled at observing patterns in nature, analyzing biological data, \
and explaining complex processes like evolution, genetics, physiology, and ecology. \
You can clearly communicate how life functions and adapts, \
and you make connections between different biological concepts \
to answer challenging questions.

Here is a question:
{input}"""

In [27]:
prompt_infos = [
    {
        "name": "physics", 
        "description": "Good for answering questions about physics", 
        "prompt_template": physics_template
    },
    {
        "name": "math", 
        "description": "Good for answering math questions", 
        "prompt_template": math_template
    },
    {
        "name": "History", 
        "description": "Good for answering history questions", 
        "prompt_template": history_template
    },
    {
        "name": "computer science", 
        "description": "Good for answering computer science questions", 
        "prompt_template": computerscience_template
    },
    {
        "name": "biology",
        "description": "Good for answering biology questions",
        "prompt_template": biology_template
    }
]

In [28]:
from langchain_classic.chains.router import MultiPromptChain
from langchain_classic.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain_core.prompts import PromptTemplate

In [29]:
llm = ChatGroq(model="llama-3.3-70b-versatile", api_key=GROQ_API_KEY, temperature=0)

In [30]:
destination_chains = {}
for p_info in prompt_infos:
    name = p_info["name"]
    prompt_template = p_info["prompt_template"]
    prompt = ChatPromptTemplate.from_template(template=prompt_template)
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain  
    
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
destinations_str = "\n".join(destinations)

In [31]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=llm, prompt=default_prompt)

In [32]:
MULTI_PROMPT_ROUTER_TEMPLATE = """Given a raw text input to a \
language model select the model prompt best suited for the input. \
You will be given the names of the available prompts and a \
description of what the prompt is best suited for. \
You may also revise the original input if you think that revising\
it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt \
names specified below OR it can be "DEFAULT" if the input is not\
well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input \
if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (remember to include the ```json)>>"""

In [33]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations=destinations_str
)
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)

router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [34]:
chain = MultiPromptChain(router_chain=router_chain, 
                         destination_chains=destination_chains, 
                         default_chain=default_chain, verbose=True
                        )

C:\Users\User\AppData\Local\Temp\ipykernel_45400\3038952769.py:1: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain = MultiPromptChain(router_chain=router_chain,


In [35]:
chain.run("What is black body radiation?")



> Entering new MultiPromptChain chain...
physics: {'input': 'What is black body radiation?'}
> Finished chain.


'Black body radiation is a fundamental concept in physics. In simple terms, it refers to the electromagnetic radiation emitted by an object at a certain temperature. The term "black body" doesn\'t mean the object is actually black, but rather that it\'s an idealized object that absorbs all the electromagnetic radiation that hits it, without reflecting or transmitting any of it.\n\nWhen an object is heated, its atoms or molecules vibrate and collide with each other, causing them to emit radiation across a wide range of wavelengths, including visible light, infrared, and ultraviolet. The distribution of this radiation is dependent on the object\'s temperature, and it follows a specific pattern, known as the black body spectrum.\n\nThe key point is that the black body spectrum is a universal curve that depends only on the temperature of the object, not on its composition or other properties. This means that any object at a given temperature will emit radiation with the same spectrum, rega

In [36]:
chain.run("what is 2 + 2")



> Entering new MultiPromptChain chain...
math: {'input': 'what is 2 + 2'}
> Finished chain.


'To answer this question, let\'s break it down into its component parts.\n\nThe question is asking for the result of adding 2 and 2 together. \n\nThe component parts of this question are:\n1. The number 2\n2. The operation of addition (+)\n3. The number 2 (again, since we\'re adding 2 + 2)\n\nNow, let\'s answer the component parts:\n1. We know the value of the number 2.\n2. We know how to perform the operation of addition, which means combining two numbers to get their total or sum.\n3. We know the value of the second number 2.\n\nNow, let\'s put the component parts together to answer the broader question:\n2 + 2 = ?\n\nTo do this, we simply add the two numbers together:\n2 + 2 = 4\n\nTherefore, the answer to the question "what is 2 + 2" is 4.'

In [37]:
chain.run("Why does every cell in our body contain DNA?")



> Entering new MultiPromptChain chain...
biology: {'input': 'Why does every cell in the human body contain DNA?'}
> Finished chain.


'The presence of DNA in every cell of the human body is a fundamental aspect of life, and it\'s essential for the proper functioning of our cells, tissues, and organs. To understand why every cell contains DNA, let\'s dive into the basics of cellular biology and genetics.\n\nDNA, or deoxyribonucleic acid, is a molecule that contains the genetic instructions for the development, growth, and function of all living organisms. It\'s often referred to as the "blueprint" or "genetic code" of life. In humans, DNA is composed of more than 3 billion base pairs, which are arranged in a specific sequence to form genes.\n\nEvery cell in the human body, from skin cells to muscle cells, contains a complete copy of the genome, which is the entire set of genetic instructions encoded in an organism\'s DNA. This is because cells are the basic building blocks of life, and they need access to the genetic information stored in DNA to perform their functions.\n\nThere are several reasons why every cell cont

**Repeat the above at least once for different inputs and chains executions - Be creative!**

In [42]:
# History question
chain.run("What were the main causes of World War I?")



> Entering new MultiPromptChain chain...
History: {'input': 'What were the main causes of World War I?'}
> Finished chain.


"The outbreak of World War I is a complex and multifaceted topic, and historians have long debated the various factors that contributed to the war. After careful consideration of the historical evidence, I would argue that the main causes of World War I can be summarized as follows:\n\n1. **Imperialism and Colonial Rivalries**: The late 19th and early 20th centuries saw a scramble for colonies and resources among European powers, particularly in Africa and Asia. This led to tensions between nations, as they competed for influence, territory, and economic dominance. The Berlin Conference of 1884-1885, which partitioned Africa among European powers, is a notable example of this phenomenon.\n\n2. **Militarism and the Arms Race**: The early 20th century witnessed a significant increase in military spending and the buildup of armed forces among European powers. This created an atmosphere of tension and competition, as nations sought to demonstrate their military prowess and preparedness for

In [43]:
# Computer science question
chain.run("What is the difference between a stack and a queue?")



> Entering new MultiPromptChain chain...
computer science: {'input': 'What is the difference between a stack and a queue in computer science?'}
> Finished chain.


"In computer science, a stack and a queue are two fundamental data structures that allow you to store and manipulate collections of elements. The primary difference between them lies in the order in which elements are added and removed.\n\n**Stack:**\nA stack is a Last-In-First-Out (LIFO) data structure, meaning that the last element added to the stack is the first one to be removed. Think of a stack of plates: when you add a new plate, you put it on top of the existing ones, and when you remove a plate, you take the top one off.\n\nHere are the basic operations you can perform on a stack:\n\n1. **Push**: Add an element to the top of the stack.\n2. **Pop**: Remove the top element from the stack.\n3. **Peek**: Look at the top element without removing it.\n\n**Queue:**\nA queue is a First-In-First-Out (FIFO) data structure, meaning that the first element added to the queue is the first one to be removed. Think of a line of people waiting for a concert: when someone new arrives, they go t

In [44]:
# Math question
chain.run("What is the integral of x^2 dx?")



> Entering new MultiPromptChain chain...
math: {'input': 'What is the integral of x^2 with respect to x?'}
> Finished chain.


"To find the integral of x^2 with respect to x, I'll break it down into its component parts.\n\nThe integral of x^2 with respect to x can be written as:\n\n∫x^2 dx\n\nTo solve this, I'll use the power rule of integration, which states that:\n\n∫x^n dx = (x^(n+1))/(n+1) + C\n\nwhere n is a constant, and C is the constant of integration.\n\nIn this case, n = 2, so I can plug that into the formula:\n\n∫x^2 dx = (x^(2+1))/(2+1) + C\n= (x^3)/3 + C\n\nTherefore, the integral of x^2 with respect to x is:\n\n∫x^2 dx = (x^3)/3 + C\n\nwhere C is the constant of integration.\n\nSo, the answer to the broader question is that the integral of x^2 with respect to x is (x^3)/3 + C."